**Authentication**

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
%load_ext google.colab.data_table

**Init**

In [3]:
project_id = 'basic-bounty-437906-r2'

from google.cloud import bigquery

client = bigquery.Client(project=project_id)

**Subqueries/ Nested Queries**

In [6]:
df = client.query('''
  SELECT
  `hash` AS transaction_hash,
  block_hash
FROM
  `bigquery-public-data.crypto_bitcoin.transactions`
WHERE
  TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-10-04")
  AND block_hash IN (
  SELECT
    `hash`
  FROM
    `bigquery-public-data.crypto_bitcoin.blocks`
  WHERE
    TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-10-04")
    AND `version` = 536870912)
LIMIT
  5''').to_dataframe()

df.head()

,transaction_hash,block_hash
0,31b89e99f517d8895ad67e90321605416c67b4d2cd2f77...,000000000000000000000aea2e3940d0b49d65d56795cf...
1,ad2a2ef73663abe1fca8efdf49bf4613c8773441c3a03a...,000000000000000000000aea2e3940d0b49d65d56795cf...
2,ef6855642971ecba5186ce544bc1f28a137f5bc4773e23...,000000000000000000000aea2e3940d0b49d65d56795cf...
3,d6f53c4bbc7582ba669575942069be32b8f2463a7a7694...,000000000000000000000aea2e3940d0b49d65d56795cf...
4,60c4fd06bb98cb1df46b875a1549e31e3a700d6d99cb48...,000000000000000000000aea2e3940d0b49d65d56795cf...


**Joins**

In [9]:
df = client.query('''
  SELECT
  t.transaction_hash,
  t.block_hash,
  tb.version
FROM (
  SELECT
    `hash` AS transaction_hash,
    block_hash
  FROM
    `bigquery-public-data.crypto_bitcoin.transactions`
  WHERE
    TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-10-04")) AS t
INNER JOIN (
  SELECT
    `hash` AS block_hash,
    `version`
  FROM
    `bigquery-public-data.crypto_bitcoin.blocks`
  WHERE
    TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-10-04")
    AND `version` = 536870912) AS tb
ON
  t.block_hash = tb.block_hash
ORDER BY
  t.transaction_hash
LIMIT
  5''').to_dataframe()

df.head()

,transaction_hash,block_hash,version
0,000082621e5d997c50e908ce1396344e9b47ad69a5f819...,0000000000000000000101ed8af9f5bca67c278accb9f0...,536870912
1,00035497117b0ccf9e5682cc7a67a6fc1fd9e54410ee0e...,0000000000000000000316aa5079c199c6cd6116a676bf...,536870912
2,0004e3fa2e7ed82d2792dda7c70b507a8f713c39b98288...,0000000000000000000101ed8af9f5bca67c278accb9f0...,536870912
3,000535226ad85d8cca363b4af74fc68662b4d2d9463a21...,0000000000000000000324f494ba5daffdac4c4abec110...,536870912
4,000634ed0482c53fe73289d2251e8cf8c07456a9e8e6d3...,0000000000000000000101ed8af9f5bca67c278accb9f0...,536870912


**Union**

In [11]:
df = client.query('''
  SELECT DISTINCT
    `hash` AS transaction_hash,
    block_hash,
    CAST(NULL AS INT64) AS version
  FROM
    `bigquery-public-data.crypto_bitcoin.transactions`
  WHERE
    TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-10-04")

  UNION DISTINCT

  SELECT DISTINCT
    CAST(NULL AS STRING) AS transaction_hash,
    `hash` AS block_hash,
    `version`
  FROM
    `bigquery-public-data.crypto_bitcoin.blocks`
  WHERE
    TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-10-04")
    AND `version` = 536870912

  ORDER BY
    block_hash
  LIMIT 5
''').to_dataframe()

df.head()


,transaction_hash,block_hash,version
0,57185dff94c3ecfdadfa1223b91b8ba19940c2a33f7160...,00000000000000000000030b706b98de3ab964946e377b...,<NA>
1,ac9a676343e9e5c715d2258ed2a93bd7d6ea8a7504c855...,00000000000000000000030b706b98de3ab964946e377b...,<NA>
2,526f3ae87d32d730984df615df935f0e444c3412b61bfe...,00000000000000000000030b706b98de3ab964946e377b...,<NA>
3,f4e9115edcc6cc510333143fa58b9501398d327c9ad62e...,00000000000000000000030b706b98de3ab964946e377b...,<NA>
4,2254bf0d6834cfd0c683ffa4e552eb73171199ede469de...,00000000000000000000030b706b98de3ab964946e377b...,<NA>


**Aggregate Function**

In [12]:
df = client.query('''
  SELECT
  block_hash,
  COUNT(`hash`) AS total_transaction
FROM
  `bigquery-public-data.crypto_bitcoin.transactions`
WHERE
  TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-10-04")
  AND block_hash IN (
  SELECT
    `hash`
  FROM
    `bigquery-public-data.crypto_bitcoin.blocks`
  WHERE
    TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-10-04"))
GROUP BY
  (block_hash)
LIMIT 5''').to_dataframe()

df.head()

,block_hash,total_transaction
0,00000000000000000001d71e919a6b79a5c0c24353f9c7...,2814
1,00000000000000000003118acd10b017b357d6ca2cb26c...,3402
2,00000000000000000002c0324c99d25df85c39deb7b544...,2810
3,00000000000000000000059f33280f9f4e7e567e2f0768...,3219
4,00000000000000000002b09c74a55188adbae267d5a206...,2433


**Window Function**

In [13]:
df = client.query('''
SELECT
  block_hash,
  total_transaction,
  ROW_NUMBER() OVER (ORDER BY total_transaction DESC) AS rank
FROM (
  SELECT
    block_hash,
    COUNT(`hash`) AS total_transaction
  FROM
    `bigquery-public-data.crypto_bitcoin.transactions`
  WHERE
    TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-08-06")
    AND block_hash IN (
    SELECT
      `hash`
    FROM
      `bigquery-public-data.crypto_bitcoin.blocks`
    WHERE
      TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-08-06"))
  GROUP BY
    block_hash
)
LIMIT 5
''').to_dataframe()

df.head()


,block_hash,total_transaction,rank
0,00000000000000000001cf7fd65d7cac4bab533ca3bcfb...,7855,1
1,000000000000000000020686ab68363ebaadd5d87c9cf1...,7402,2
2,000000000000000000018028ae422d7b637ec8c41ff094...,7239,3
3,0000000000000000000053d7f74913d44db8f74727b321...,6836,4
4,000000000000000000021fcc083737c58b717699815348...,6828,5


**Common Table Expressions**

In [14]:
df = client.query('''
WITH transactions_cte AS (
  -- CTE for transactions table
  SELECT
    `hash` AS transaction_hash,
    block_hash
  FROM
    `bigquery-public-data.crypto_bitcoin.transactions`
  WHERE
    TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-10-04")
),
blocks_cte AS (
  SELECT
    `hash` AS block_hash,
    `version`
  FROM
    `bigquery-public-data.crypto_bitcoin.blocks`
  WHERE
    TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-10-04")
    AND `version` = 536870912
)

SELECT
  t.transaction_hash,
  t.block_hash,
  b.version
FROM
  transactions_cte AS t
INNER JOIN
  blocks_cte AS b
ON
  t.block_hash = b.block_hash
ORDER BY
  t.transaction_hash
LIMIT
  5
''').to_dataframe()

df.head()


,transaction_hash,block_hash,version
0,000082621e5d997c50e908ce1396344e9b47ad69a5f819...,0000000000000000000101ed8af9f5bca67c278accb9f0...,536870912
1,00035497117b0ccf9e5682cc7a67a6fc1fd9e54410ee0e...,0000000000000000000316aa5079c199c6cd6116a676bf...,536870912
2,0004e3fa2e7ed82d2792dda7c70b507a8f713c39b98288...,0000000000000000000101ed8af9f5bca67c278accb9f0...,536870912
3,000535226ad85d8cca363b4af74fc68662b4d2d9463a21...,0000000000000000000324f494ba5daffdac4c4abec110...,536870912
4,000634ed0482c53fe73289d2251e8cf8c07456a9e8e6d3...,0000000000000000000101ed8af9f5bca67c278accb9f0...,536870912


**Pivot**

In [15]:
%%bigquery --project basic-bounty-437906-r2

WITH mock_transactions AS (
  SELECT "2024-10-01" AS transaction_date, "buy" AS transaction_type, 100 AS amount UNION ALL
  SELECT "2024-10-01", "sell", 200 UNION ALL
  SELECT "2024-10-01", "transfer", 50 UNION ALL
  SELECT "2024-10-02", "buy", 300 UNION ALL
  SELECT "2024-10-02", "sell", 150 UNION ALL
  SELECT "2024-10-02", "transfer", 100 UNION ALL
  SELECT "2024-10-03", "buy", 400 UNION ALL
  SELECT "2024-10-03", "sell", 250 UNION ALL
  SELECT "2024-10-03", "transfer", 150
)

SELECT *
FROM (
  SELECT
    transaction_date,
    transaction_type,
    amount
  FROM
    mock_transactions
)
PIVOT (
  SUM(amount) FOR transaction_type IN ('buy', 'sell', 'transfer')
)
ORDER BY
  transaction_date

Query is running:   0%|          |

Downloading:   0%|          |

,transaction_date,buy,sell,transfer
0,2024-10-01,100,200,50
1,2024-10-02,300,150,100
2,2024-10-03,400,250,150


**String Manipulation (Concat)**

In [19]:
%%bigquery --project basic-bounty-437906-r2
SELECT
  DISTINCT CONCAT(timestamp_month,' Count: ', transaction_count) AS address
FROM
  `bigquery-public-data.crypto_bitcoin.blocks`
LIMIT
  5

Query is running:   0%|          |

Downloading:   0%|          |

,address
0,2020-10-01 Count: 1736
1,2020-10-01 Count: 2048
2,2020-10-01 Count: 2753
3,2020-10-01 Count: 2711
4,2020-10-01 Count: 2547


**Date and Time**

In [21]:
%%bigquery --project basic-bounty-437906-r2
  SELECT
    TIMESTAMP_TRUNC(block_timestamp, DAY) AS tx_date
  FROM
    bigquery-public-data.crypto_bitcoin.transactions
  WHERE
    block_timestamp >= CAST(DATE_SUB(CURRENT_DATE(), INTERVAL 10 DAY) AS TIMESTAMP)
  LIMIT
    5

Query is running:   0%|          |

Downloading:   0%|          |

,tx_date
0,2024-10-07 00:00:00+00:00
1,2024-10-07 00:00:00+00:00
2,2024-10-07 00:00:00+00:00
3,2024-10-07 00:00:00+00:00
4,2024-10-07 00:00:00+00:00


In [22]:
df = client.query('''
  SELECT
    block_hash,
    size,
    CASE
      WHEN size > 500 THEN 'big'
      ELSE 'small'
    END AS category
  FROM
    `bigquery-public-data.crypto_bitcoin.transactions`
  WHERE
    TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-10-04")
  LIMIT 5
''').to_dataframe()

df.head()


,block_hash,size,category
0,00000000000000000002c123e9534094fa4278e9a8b7a3...,332,small
1,00000000000000000002c123e9534094fa4278e9a8b7a3...,959,big
2,00000000000000000002c123e9534094fa4278e9a8b7a3...,671,big
3,00000000000000000002c123e9534094fa4278e9a8b7a3...,342,small
4,00000000000000000002c123e9534094fa4278e9a8b7a3...,545,big


**User Defined Function**



In [25]:
%%bigquery --project basic-bounty-437906-r2

CREATE TEMP FUNCTION getCategory(size INT64) AS (
  CASE
    WHEN size > 500 THEN 'big'
    ELSE 'small'
  END
);

SELECT
  block_hash,
  size,
  getCategory(size) AS category
FROM
  `bigquery-public-data.crypto_bitcoin.transactions`
WHERE
  TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-10-04")
LIMIT 5;


Query is running:   0%|          |

Downloading:   0%|          |

,block_hash,size,category
0,00000000000000000002c123e9534094fa4278e9a8b7a3...,214,small
1,00000000000000000002c123e9534094fa4278e9a8b7a3...,340,small
2,00000000000000000002c123e9534094fa4278e9a8b7a3...,521,big
3,00000000000000000002c123e9534094fa4278e9a8b7a3...,321,small
4,00000000000000000002c123e9534094fa4278e9a8b7a3...,284,small


****

**Recursive Queries**

In [29]:
%%bigquery --project basic-bounty-437906-r2

WITH RECURSIVE numbers AS (
  SELECT 1 AS num
  UNION ALL
  SELECT num + 1
  FROM numbers
  WHERE num < 5
)

SELECT num
FROM numbers
ORDER BY 1


Query is running:   0%|          |

Downloading:   0%|          |

,num
0,1
1,2
2,3
3,4
4,5
